### NNDL baseline trainings

#### Download the kaggle datasets

In [1]:
#Download the dataset from Dropbox
!wget -O released_data.zip "https://www.dropbox.com/s/c2dvapqb613ukhw/Released_Data-20221201T215316Z-001.zip?dl=0"

#Unzip the train, test and other datasets
!unzip -q released_data.zip
!unzip -q ./Released_Data/test_shuffle.zip
!unzip -q ./Released_Data/train_shuffle.zip

--2022-12-13 23:09:31--  https://www.dropbox.com/s/c2dvapqb613ukhw/Released_Data-20221201T215316Z-001.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.66.18, 2620:100:6021:18::a27d:4112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.66.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/c2dvapqb613ukhw/Released_Data-20221201T215316Z-001.zip [following]
--2022-12-13 23:09:32--  https://www.dropbox.com/s/raw/c2dvapqb613ukhw/Released_Data-20221201T215316Z-001.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc90af0e2df3da57359b424cb9bf.dl.dropboxusercontent.com/cd/0/inline/Byl-K3uQSveIArKqb1Q1U98Gtguh8A7eJQJVZiNgvZYw9-DlXrGtoQ4pNjYdyTSNSRUB5xc8jY4ZE-Tcg45eYsvA7lrd7Y2XczKTYXwQw2fduA3Nd133JIyM2oZ8hJJknkzUID_TCzcR7MUUcDJ4A0bTR5jhnxCtymihEkW28k_ymA/file# [following]
--2022-12-13 23:09:32--  https://uc90af0e2df3da57359b424cb9bf.dl.dropboxusercontent.com/cd/0/i

#### Dataset and Dataloader preparation

##### Train and Validation data splits

In [41]:
import os
import sys
import numpy as np
import pandas as pd

full_imgs = np.array(os.listdir('./train_shuffle'))
full_labels = pd.read_csv('./Released_Data/train_data.csv').to_numpy()

labels_list = [set(),set(),set()]

#prepare lists based on the super classes
labels_list = [{}, {}, {}]
val_labels_list = [{}, {}, {}]

for i in range(len(full_labels)):
    if full_labels[i][2] in labels_list[full_labels[i][1]]:
        labels_list[full_labels[i][1]][full_labels[i][2]] += 1
    else:
        labels_list[full_labels[i][1]][full_labels[i][2]] = 1
        val_labels_list[full_labels[i][1]][full_labels[i][2]] = 0

# Check the number of times each sub class appears
# print(dict(sorted(labels_list[0].items(), key=lambda item: item[1])))
# print(dict(sorted(labels_list[1].items(), key=lambda item: item[1])))
# print(dict(sorted(labels_list[2].items(), key=lambda item: item[1])))

#Splitting the train and validation datasets (90 - 10 split)
train_imgs = []
val_imgs = []

for i in range(len(full_labels)):
    if val_labels_list[full_labels[i][1]][full_labels[i][2]] <= (0.1)*(labels_list[full_labels[i][1]][full_labels[i][2]]):
        val_labels_list[full_labels[i][1]][full_labels[i][2]] += 1
        val_imgs.append(full_labels[i][0])
    else:
        train_imgs.append(full_labels[i][0])

#Check if there is any common image between training and validation datasets
common_names = [name for name in train_imgs if name in val_imgs]
print("Common names: ", common_names)

train_data = full_labels[np.isin(full_labels[:, 0], train_imgs)]
val_data = full_labels[np.isin(full_labels[:, 0], val_imgs)]

#Check if there is any name missing from the training dataset
ver_train_names = train_data[:, 0]
error_names = [name for name in ver_train_names if name not in train_imgs]
print("Error names: ", error_names)

print("Total number of images: ", len(full_imgs))
print("Number of training images: ", train_data.shape)
print("Number of validation images: ", val_data.shape)
# print(type(train_data[0][0]))
# print(type(train_data[0][1]))
# print(type(val_data[0][0]))
# print(type(val_data[0][1]))


# --------Code to check the distribution of super and sub classes in the training and validation datasets---------
# train_sub_dict = {}
# train_super_dict = {}
# val_sub_dict = {}
# val_super_dict = {}

# for i in range(full_labels.shape[0]):
#     name = full_labels[i][0]
#     super_class = full_labels[i][1]
#     sub_class = full_labels[i][2]

#     if name in val_data[:, 0]:
#         if super_class in val_super_dict:
#             val_super_dict[super_class] += 1
#         else:
#             val_super_dict[super_class] = 1
        
#         if sub_class in val_sub_dict:
#             val_sub_dict[sub_class] += 1
#         else:
#             val_sub_dict[sub_class] = 1
#     else:
#         if super_class in train_super_dict:
#             train_super_dict[super_class] += 1
#         else:
#             train_super_dict[super_class] = 1
        
#         if sub_class in train_sub_dict:
#             train_sub_dict[sub_class] += 1
#         else:
#             train_sub_dict[sub_class] = 1

# print("Super Class distribution:")
# for k in val_super_dict:
#     print("Class: ", k, " Validation: ", val_super_dict[k], " Train: ", train_super_dict[k], " Ratio: ", train_super_dict[k]/val_super_dict[k])

# print("------------------------------")
# print("Sub Class distribution:")
# min_val = 10000
# max_val = -1
# for k in val_sub_dict:
#     min_val = min(min_val, train_sub_dict[k]/val_sub_dict[k])
#     max_val = max(max_val, train_sub_dict[k]/val_sub_dict[k])
#     print("Class: ", k, " Validation: ", val_sub_dict[k], " Train: ", train_sub_dict[k], " Ratio: ", train_sub_dict[k]/val_sub_dict[k])

# print("Minimum Sub Class ratio: ", min_val)
# print("Maximum Sub Class ratio: ", max_val)



Common names:  []
Error names:  []
Total number of images:  6472
Number of training images:  (5738, 3)
Number of validation images:  (734, 3)


##### Information about the images in the dataset

In [42]:
import os
import sys
import cv2
from PIL import Image
from google.colab.patches import cv2_imshow

full_test_imgs = os.listdir("./test_shuffle")

train_img = cv2.imread(os.path.join('train_shuffle', train_data[0][0]), cv2.IMREAD_UNCHANGED)
print(train_data[0][0], train_img.shape)
cv2_imshow(train_img)

val_img = cv2.imread(os.path.join('train_shuffle', val_data[0][0]), cv2.IMREAD_UNCHANGED)
print(val_data[0][0], val_img.shape)
cv2_imshow(val_img)

test_img = cv2.imread(os.path.join('train_shuffle', full_test_imgs[0]), cv2.IMREAD_UNCHANGED)
print(full_test_imgs[0], test_img.shape)
cv2_imshow(test_img)


328.jpg (8, 8, 3)


0.jpg (8, 8, 3)


4713.jpg (8, 8, 3)


##### KaggleDataset

In [43]:
import os
import sys
import pandas as pd
import cv2
import torch
import torch.nn as nn

import albumentations as A
from albumentations.pytorch import ToTensorV2

class KaggleDataset(nn.Module):
    def __init__(self, mode = 'train'):
        super(KaggleDataset, self).__init__()
        self.mode = mode

        if mode == 'train':
            self.dataset = train_data
        else:
            self.dataset = val_data
    
    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        img_name = self.dataset[idx][0]
        img = cv2.imread(os.path.join('train_shuffle', img_name))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)    #convert image from BGR to RGB format

        super_class = torch.tensor(self.dataset[idx][1], dtype = torch.float32)
        sub_class = torch.tensor(self.dataset[idx][2], dtype = torch.float32)

        apply_transform = self.transform_data()
        image = apply_transform(image = img)['image']

        return image, super_class, sub_class

    def transform_data(self):

        if self.mode == 'train':
            transform_data = A.Compose(
              [
                  #always resize the image to 329x224
                  #A.Resize(height = 329, width= 224, interpolation = cv2.INTER_AREA, p=1),
                  A.HorizontalFlip(p=0.4),  
                  A.ShiftScaleRotate(shift_limit=0.025, scale_limit=0, rotate_limit=15, p=0.5),
                  #A.RandomCrop(height = 224, width = 224, p=1),
                  #randomly change brightness, contrast, and saturation of the image 50% of the time
                  A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue = 0, p=0.5), 
                  A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, p=1), 
                  ToTensorV2(p=1),
              ])
        else:     #augmentations during validation and testing
          transform_data = A.Compose(
          [
              #always resize the image to 329x224
              #A.Resize(height = 329, width = 224, p=1),   
              A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, p=1),
              ToTensorV2(p=1),
          ])
    
        return transform_data

##### KaggleDataloader

In [46]:
from torch.utils.data import DataLoader

train_dataset = KaggleDataset(mode='train')
val_dataset = KaggleDataset(mode='val')
    
train_loader = DataLoader(dataset = train_dataset, batch_size = 64, shuffle = True, num_workers = 8, pin_memory = True)
val_loader = DataLoader(dataset = val_dataset, batch_size = 64, shuffle = False)

# print(len(train_loader))
# print(len(val_loader))

90
12
